# Analyzing fire department payrolls in California: 2011-2018

This notebook analyzes annual government payroll [data](https://publicpay.ca.gov/Reports/RawExport.aspx) compiled and released annually by the California state controller's office. The data include anonymized salary information for all employees at cities, counties, special districts and state government. 

---

### Load python tools

In [134]:
import pandas as pd
import geopandas as gpd
import cpi
from urllib.request import urlopen 
import pyarrow
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
alt.renderers.enable('notebook')
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format

### Import fire payroll data

In [135]:
# processed in 01-california-payroll-descriptives
agency_ot = pd.read_csv('output/agency_overtime.csv')
agency_ot_share = pd.read_csv('output/agency_overtime_share_ot.csv')

In [142]:
la.head(1)

,index,year,type,population,employer,department,position,overtime,wages,benefits,employercounty,basewages,adjusted_overtime,adjusted_wages,adjusted_benefits,adjusted_basewages,fire,employerfull
970571,970571,2018,CITY,"4,040,079",LOS ANGELES,FIRE,APPARATUS OPERATOR,"27,902","145,581","70,947",LOS ANGELES,"117,679","27,902","145,581","70,947","117,679",True,LOS_ANGELES_CITY


---

### What about Los Angeles city and county fire?

In [146]:
la = payroll_fire[payroll_fire['employer'] == 'LOS ANGELES']

In [151]:
la.groupby(['employerfull', 'year']).agg({'adjusted_overtime':'sum', 'adjusted_basewages':'sum'}).reset_index()

,employerfull,year,adjusted_overtime,adjusted_basewages
0,LOS_ANGELES_CITY,2011,"106,442,381","396,621,835"
1,LOS_ANGELES_CITY,2012,"122,766,643","377,939,538"
2,LOS_ANGELES_CITY,2013,"150,689,736","372,974,009"
3,LOS_ANGELES_CITY,2014,"170,170,047","333,744,338"
4,LOS_ANGELES_CITY,2015,"194,577,723","376,086,417"
5,LOS_ANGELES_CITY,2016,"193,805,688","378,143,097"
6,LOS_ANGELES_CITY,2017,"200,297,491","399,058,793"
7,LOS_ANGELES_CITY,2018,"194,037,539","399,924,796"
8,LOS_ANGELES_COUNTY,2011,"141,609,864","422,086,794"
9,LOS_ANGELES_COUNTY,2012,"146,737,872","415,243,036"


---

### Create a dataframe combining and pivoting listing all the large places

In [169]:
large_places_df = payroll_fire[(payroll_fire['population'] > 10000) \
                               & (payroll_fire['employer'] != 'FRESNO')].groupby(['employer', 'type', 'year'])\
                .agg({'adjusted_overtime': 'sum'}).reset_index()

In [170]:
large_places_df_pivot = pd.DataFrame(pd.pivot_table(large_places_df, \
                values='adjusted_overtime', index=['employer', 'type'], columns=['year']).reset_index().fillna(0))

In [171]:
large_places_df_pivot['adjusted_ot_pct_change'] = \
                ((large_places_df_pivot['2018']-large_places_df_pivot['2011'])/\
                 large_places_df_pivot['2011'])*100

In [172]:
places = large_places_df_pivot[(large_places_df_pivot['2011'] > 10000) & (large_places_df_pivot['2018'] > 10000)]

In [173]:
large_places_df_pivot.head()

year,employer,type,2011,2012,2013,2014,2015,2016,2017,2018,adjusted_ot_pct_change
0,ALAMEDA,CITY,"606,326","413,811","480,147","938,339","1,721,621","1,007,731","1,057,992","1,412,566",133
1,ALAMEDA,COUNTY,0,0,0,0,"13,956,109","15,865,394","3,441,105","16,881,483",inf
2,ALBANY,CITY,"571,784","770,969","638,364","604,750","649,010",0,"794,677","924,343",62
3,ALHAMBRA,CITY,"2,379,257","2,574,230","2,434,378","2,503,066","2,175,470","2,959,928","3,442,536","3,713,556",56
4,AMADOR,COUNTY,0,0,0,0,0,0,"501,099","461,893",inf


In [174]:
large_places_df_pivot['adjusted_ot_pct_change'] = \
                ((large_places_df_pivot['2018']-large_places_df_pivot['2011'])/\
                 large_places_df_pivot['2014'])*100

In [175]:
places = large_places_df_pivot[(large_places_df_pivot['2014'] > 0) & (large_places_df_pivot['2018'] > 0)]

In [176]:
places.sort_values(by='2018', ascending=False).head()

year,employer,type,2011,2012,2013,2014,2015,2016,2017,2018,adjusted_ot_pct_change
91,LOS ANGELES,COUNTY,"140,482,851","145,158,300","160,012,219","163,180,317","176,424,573","198,041,672","203,532,044","201,820,125",38
90,LOS ANGELES,CITY,"106,442,381","122,766,643","150,689,736","170,170,047","194,577,723","193,805,688","200,297,491","194,037,539",51
157,SAN DIEGO,CITY,"21,771,260","984,990","33,931,223","34,051,944","32,620,927","34,626,760","39,584,415","43,235,824",63
158,SAN FRANCISCO,CITY,"32,605,597","44,103,503","47,549,997","40,190,776","42,090,738","43,809,245","35,791,590","34,676,006",5
204,VENTURA,COUNTY,"17,116,105","16,796,105","17,391,128","17,945,102","19,650,785","21,168,536","23,244,233","23,703,542",37


In [177]:
places.to_csv('output/places/large_places.csv')

---

Data source: https://publicpay.ca.gov/Reports/RawExport.aspx